In [1]:
import requests
import json
import pandas as pd

In [69]:
import requests
import pandas as pd

# ID der OpenSenseMap-Box
id = '5c2b8d6e2c801000191443f9'
url = "https://api.opensensemap.org/boxes/" + id

# Anfrage an die API
response = requests.get(url)
data = response.json()

# Extrahieren der relevanten Daten
box_name = data.get('name', 'Unknown')
coordinates = data.get('currentLocation', {}).get('coordinates', [None, None])
sensors = data.get('sensors', [])

# Erstellen einer Liste von Dictionaries mit den Sensordaten
sensor_data = []
for sensor in sensors:
    sensor_info = {
        'Sensor ID': sensor.get('_id', 'Unknown'),
        'Type': sensor.get('sensorType', 'Unknown'),
        'Title': sensor.get('title', 'Unknown'),
        'Unit': sensor.get('unit', 'Unknown'),
        'Last Measurement Value': sensor.get('lastMeasurement', {}).get('value', 'Unknown'),
        'Last Measurement Timestamp': sensor.get('lastMeasurement', {}).get('createdAt', 'Unknown'),
        'Latitude': coordinates[1],
        'Longitude': coordinates[0]
    }
    sensor_data.append(sensor_info)

# Erstellen eines DataFrames aus der Liste der Sensordaten
df = pd.DataFrame(sensor_data)

# Ausgabe des DataFrames
print(f"Data from OpenSenseMap Box: {box_name}")
df


Data from OpenSenseMap Box: florenz.ka.balkon


,Sensor ID,Type,Title,Unit,Last Measurement Value,Last Measurement Timestamp,Latitude,Longitude
0,5c2b8d6e2c801000191443fe,SDS 011,PM10,µg/m³,1.50,2024-06-16T20:32:43.729Z,49.00662,8.513592
1,5c2b8d6e2c801000191443fd,SDS 011,PM2.5,µg/m³,1.30,2024-06-16T20:32:43.729Z,49.00662,8.513592
2,5c2b8d6e2c801000191443fc,BME280,Temperatur,°C,19.22,2024-06-16T20:32:43.729Z,49.00662,8.513592
3,5c2b8d6e2c801000191443fb,BME280,rel. Luftfeuchte,%,78.37,2024-06-16T20:32:43.729Z,49.00662,8.513592
4,5c2b8d6e2c801000191443fa,BME280,Luftdruck,Pa,99596.53,2024-06-16T20:32:43.729Z,49.00662,8.513592


In [ ]:
import requests
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# ID der OpenSenseMap-Box
id = ['5c2b8d6e2c801000191443f9', '5e163c0f7e9052001a4a9449', '5a36e1e91ed947001059d891', '5dde9523ba7944001da4150e', '5ac38bcf850005001bfd2559', '5e0d12642ecdf1001a3343fa', '5c2b8d6e2c801000191443f9', '665ca5828e1be00008cce9b7', '6252afcfd7e732001bb6b9f7', '6645db6eeb5aad0007a6e4b6', '6645e1c8eb5aad0007b226b5', '5abeb022850005001b51d92f', '598b2f86e3b1fa0010f0f8d4', '5ac1e08f850005001bc13fac', '5ba8c29de624740019faf133']
url = "https://api.opensensemap.org/boxes/" + id

# Anfrage an die API
response = requests.get(url)
data = response.json()

# Extrahieren der relevanten Daten
box_name = data.get('name', 'Unknown')
coordinates = data.get('currentLocation', {}).get('coordinates', [None, None])
sensors = data.get('sensors', [])

# Erstellen einer Liste von Dictionaries mit den Sensordaten
sensor_data = []
for sensor in sensors:
    sensor_info = {
        'Sensor ID': sensor.get('_id', 'Unknown'),
        'Type': sensor.get('sensorType', 'Unknown'),
        'Title': sensor.get('title', 'Unknown'),
        'Unit': sensor.get('unit', 'Unknown'),
        'Last Measurement Value': float(sensor.get('lastMeasurement', {}).get('value', 0)),
        'Last Measurement Timestamp': sensor.get('lastMeasurement', {}).get('createdAt', 'Unknown'),
        'Latitude': coordinates[1],
        'Longitude': coordinates[0]
    }
    sensor_data.append(sensor_info)

# Erstellen eines DataFrames aus der Liste der Sensordaten
df = pd.DataFrame(sensor_data)

# Initialisieren der Dash-App
app = Dash(__name__)

# Layout der Dash-App
app.layout = html.Div([
    html.H1(f"Data from OpenSenseMap Box: {box_name}"),
    dcc.Dropdown(
        id='title-selector',
        options=[{'label': title, 'value': title} for title in df['Title'].unique()],
        value=df['Title'].unique()[0],
        clearable=False
    ),
    dcc.Graph(id='map')
])

# Callback-Funktion zur Aktualisierung des Scatter Plots
@app.callback(
    Output('map', 'figure'),
    [Input('title-selector', 'value')]
)
def update_map(selected_title):
    filtered_df = df[df['Title'] == selected_title]
    fig = px.scatter_mapbox(filtered_df, lat="Latitude", lon="Longitude", color="Last Measurement Value",
                            size="Last Measurement Value", hover_name="Title", hover_data=["Type", "Unit", "Last Measurement Value"],
                            color_continuous_scale=px.colors.sequential.Bluered, size_max=15, zoom=10)

    fig.update_layout(mapbox_style="carto-positron")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    return fig

# Starten der Dash-App
if __name__ == '__main__':
    app.run_server(debug=True)


In [6]:
import requests
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# IDs der OpenSenseMap-Boxen
#ids = [
 #   '5c2b8d6e2c801000191443f9', '5e163c0f7e9052001a4a9449', '5a36e1e91ed947001059d891', 
  #  '5dde9523ba7944001da4150e', '5e0d12642ecdf1001a3343fa', 
   # '5c2b8d6e2c801000191443f9', '665ca5828e1be00008cce9b7', '6252afcfd7e732001bb6b9f7', 
    #'6645db6eeb5aad0007a6e4b6', '6645e1c8eb5aad0007b226b5', '5abeb022850005001b51d92f', 
    #'5ac1e08f850005001bc13fac'
#]


ids = ['5abeb022850005001b51d92f', '5c2b8d6e2c801000191443f9', '5e0d12642ecdf1001a3343fa', '5dde9523ba7944001da4150e', '5dde9523ba7944001da4150e']
# Funktion zum Abrufen und Verarbeiten der Daten
def fetch_sensebox_data(ids):
    all_sensor_data = []
    for id in ids:
        url = "https://api.opensensemap.org/boxes/" + id
        response = requests.get(url)
        data = response.json()
        
        box_name = data.get('name', 'Unknown')
        coordinates = data.get('currentLocation', {}).get('coordinates', [None, None])
        sensors = data.get('sensors', [])
        
        for sensor in sensors:
            sensor_info = {
                'Box ID': id,
                'Box Name': box_name,
                'Sensor ID': sensor.get('_id', 'Unknown'),
                'Type': sensor.get('sensorType', 'Unknown'),
                'Title': sensor.get('title', 'Unknown'),
                'Unit': sensor.get('unit', 'Unknown'),
                'Last Measurement Value': float(sensor.get('lastMeasurement', {}).get('value', 0)),
                'Last Measurement Timestamp': sensor.get('lastMeasurement', {}).get('createdAt', 'Unknown'),
                'Latitude': coordinates[1],
                'Longitude': coordinates[0]
            }
            all_sensor_data.append(sensor_info)
    
    return pd.DataFrame(all_sensor_data)

# Abrufen der Daten
df = fetch_sensebox_data(ids)
# df['Title'] = df['Title'].str.replace('Temperature', 'Temperatur')
# df['Title'] = df['Title'].str.replace('Humidity', 'rel. Luftfeuchte')

# Initialisieren der Dash-App
app = Dash(__name__)

# Layout der Dash-App
app.layout = html.Div([
    html.H1("OpenSenseMap Data"),
    dcc.Dropdown(
        id='title-selector',
        options=[{'label': title, 'value': title} for title in df['Title'].unique()],
        value=df['Title'].unique()[0],
        clearable=False
    ),
    dcc.Graph(id='map')
])

# Callback-Funktion zur Aktualisierung des Scatter Plots
@app.callback(
    Output('map', 'figure'),
    [Input('title-selector', 'value')]
)
def update_map(selected_title):
    filtered_df = df[df['Title'] == selected_title]
    fig = px.scatter_mapbox(filtered_df, lat="Latitude", lon="Longitude", color="Last Measurement Value",
                            size="Last Measurement Value", hover_name="Title", hover_data=["Box Name", "Type", "Unit", "Last Measurement Value"],
                            color_continuous_scale=px.colors.sequential.Bluered, size_max=15, zoom=10)
    
    fig.update_layout(mapbox_style="carto-positron")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    return fig

# Starten der Dash-App
if __name__ == '__main__':
    app.run_server(debug=True)


In [9]:
df['Sensor ID']

0     5c2b8d6e2c801000191443fe
1     5c2b8d6e2c801000191443fd
2     5c2b8d6e2c801000191443fc
3     5c2b8d6e2c801000191443fb
4     5c2b8d6e2c801000191443fa
                ...           
62    5abeb022850005001b51d930
63    5ac1e08f850005001bc13fb0
64    5ac1e08f850005001bc13faf
65    5ac1e08f850005001bc13fae
66    5ac1e08f850005001bc13fad
Name: Sensor ID, Length: 67, dtype: object

In [ ]:
# replace Humidity with rel. Luftfeuchte
df['Title'] = df['Title'].str.replace('Humidity', 'rel. Luftfeuchte')
df['Title'] = df['Title'].str.replace('Temperature', 'Temperatur')

In [6]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

In [62]:
url = "https://api.opensensemap.org/boxes/6252afcfd7e732001bb6b9f7"

response = requests.get(url)
sensor = response.json()

In [ ]:
sensor

In [132]:
sensor = pd.DataFrame(sensor['sensors'])

In [146]:
import requests
import json
from datetime import datetime, timedelta

sensebox_id = '6252afcfd7e732001bb6b9f7'
sensor_id = '6252afcfd7e732001bb6b9fb'
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 6, 20)
all_data = []

def get_data_chunk(sensebox_id, sensor_id, from_date, to_date):
    url = f'https://api.opensensemap.org/boxes/{sensebox_id}/data/{sensor_id}?from-date={from_date}&to-date={to_date}&download=true&format=json'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Fehler bei der Anfrage: {response.status_code}')
        return []

current_start_date = start_date

while current_start_date < end_date:
    current_end_date = current_start_date + timedelta(days=7)  # Zeitraum von 7 Tagen
    if current_end_date > end_date:
        current_end_date = end_date

    from_date_str = current_start_date.isoformat() + 'Z'
    to_date_str = current_end_date.isoformat() + 'Z'

    data_chunk = get_data_chunk(sensebox_id, sensor_id, from_date_str, to_date_str)
    all_data.extend(data_chunk)

    current_start_date = current_end_date

# Speichern der Daten in einer JSON-Datei
with open('sensebox_data.json', 'w') as f:
    json.dump(all_data, f)

print(f'Es wurden {len(all_data)} Messwerte abgerufen und gespeichert.')


In [83]:
all_data

[{'value': '9.28',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:58:45.645Z'},
 {'value': '9.38',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:53:57.531Z'},
 {'value': '9.39',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:48:58.348Z'},
 {'value': '9.35',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:43:59.469Z'},
 {'value': '9.39',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:39:00.395Z'},
 {'value': '9.41',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:34:01.746Z'},
 {'value': '9.46',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:29:01.960Z'},
 {'value': '9.52',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:24:02.380Z'},
 {'value': '9.6',
  'location': [8.425253, 49.014943, 115],
  'createdAt': '2022-04-15T23:19:02.656Z'},
 {'value': '9.63',
  'location': [8.425253, 49.014943, 1

In [73]:
data.drop(columns='location', inplace=True)
data['createdAt'] = pd.to_datetime(data['createdAt'])
data['values'] = data['value'].astype(float)
data.sort_values(by='createdAt', inplace=True)

In [15]:
import requests
import pandas as pd
from datetime import datetime, timedelta

sensebox_id = '6252afcfd7e732001bb6b9f7'
sensor_ids = [
    '6252afcfd7e732001bb6b9fb', '6252afcfd7e732001bb6b9fa', 
    '6252afcfd7e732001bb6b9f9', '6252afcfd7e732001bb6b9f8', 
    '6252bc0fd7e732001bbcacbb', '6252bc0fd7e732001bbcacbc', 
    '6252bc0fd7e732001bbcacbd', '6415a8fcc383840007fecd71', 
    '6415b84ec38384000706cb0c'
]
# start_date = datetime(2022, 1, 1)
start_data = datetime(2024, 6, 10)
end_date = datetime(2024, 6, 20)

def get_data_chunk(sensebox_id, sensor_id, from_date, to_date):
    url = f'https://api.opensensemap.org/boxes/{sensebox_id}/data/{sensor_id}?from-date={from_date}&to-date={to_date}&download=true&format=json'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Fehler bei der Anfrage: {response.status_code}')
        return []

def fetch_sensor_data(sensebox_id, sensor_id, start_date, end_date):
    current_start_date = start_date
    sensor_data = []
    
    while current_start_date < end_date:
        current_end_date = current_start_date + timedelta(days=7)  # 7-Tage-Intervall
        if current_end_date > end_date:
            current_end_date = end_date

        from_date_str = current_start_date.isoformat() + 'Z'
        to_date_str = current_end_date.isoformat() + 'Z'

        data_chunk = get_data_chunk(sensebox_id, sensor_id, from_date_str, to_date_str)
        sensor_data.extend(data_chunk)

        current_start_date = current_end_date

    return sensor_data

# Abrufen und Speichern der Daten in DataFrames
dataframes = {}

for sensor_id in sensor_ids:
    sensor_data = fetch_sensor_data(sensebox_id, sensor_id, start_date, end_date)
    df = pd.DataFrame(sensor_data)[['createdAt', 'value']]
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df = df.rename(columns={'value': sensor_id})
    dataframes[sensor_id] = df

# Zusammenführen der DataFrames auf Basis der 'createdAt'-Spalte
main_df = None

for sensor_id, df in dataframes.items():
    if main_df is None:
        main_df = df
    else:
        main_df = pd.merge(main_df, df, on='createdAt', how='outer')

# Sortieren nach 'createdAt' und Speichern in einer Datei
main_df['createdAt'] = pd.to_datetime(main_df['createdAt'])
main_df = main_df.sort_values('createdAt')

# Speichern des DataFrames in einer CSV-Datei
# main_df.to_csv('merged_sensor_data.csv', index=False)

print(f'Daten wurden erfolgreich abgerufen.')


KeyboardInterrupt: 

In [ ]:
main_df

In [ ]:
https://api.opensensemap.org//boxes/data?boxid=6252afcfd7e732001bb6b9f7&columns=boxName,unit,value,createdAt&download=true&format=csv&from-date=2024-06-16T22:00:00.000Z&phenomenon=Temperature&to-date=2024-06-19T22:00:00.000Z

In [30]:
import requests

# URL der API
url = "https://api.opensensemap.org//boxes/data"
params = {
    "boxid": "6252afcfd7e732001bb6b9f7",
    "columns": "unit,value,createdAt",
    "download": "true",
    "format": "csv",
    "from-date": "2000-01-01T00:00:01.000Z",
    "phenomenon": "Temperature",
    "to-date": "2024-06-19T22:00:00.000Z"
}

# Anfrage an die API senden
response = requests.get(url, params=params)

# Überprüfen, ob die Anfrage erfolgreich war
if response.status_code == 200:
    # Speichern der CSV-Datei
    with open("data.csv", "wb") as file:
        file.write(response.content)
    print("CSV-Datei erfolgreich heruntergeladen und gespeichert.")
else:
    print(f"Fehler beim Herunterladen der CSV-Datei. Statuscode: {response.status_code}")


CSV-Datei erfolgreich heruntergeladen und gespeichert.


In [ ]:
data

In [21]:
url = 'https://api.opensensemap.org/boxes/6252afcfd7e732001bb6b9f7'
response = requests.get(url)
data = response.json()
sensor_titles = [sensor['title'] for sensor in data['sensors']]

['Temperature',
 'Global Radiation',
 'Rain (1h)',
 'Windspeed',
 'Humidity',
 'UV-A Radiation',
 'UV-B Radiation',
 'Wind Direction',
 'Pressure']

In [30]:
import requests
import pandas as pd
import io

# Funktion zum Abrufen der Sensortitel
def get_sensor_titles(sensebox_id):
    url = f'https://api.opensensemap.org/boxes/6252afcfd7e732001bb6b9f7'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return [{'id': sensor['_id'], 'title': sensor['title']} for sensor in data['sensors']]
    else:
        print(f'Fehler beim Abrufen der Sensor-Titel: {response.status_code}')
        return []


sensebox_id = '6252afcfd7e732001bb6b9f7'
start_date = "2024-06-18T00:00:00.000Z"
end_date = "2024-06-19T22:00:00.000Z"
sensor_titles = get_sensor_titles(sensebox_id)

def fetch_sensor_data(sensebox_id, sensor_title, start_date, end_date):
    url = "https://api.opensensemap.org//boxes/data"
    params = {
        "boxid": sensebox_id,
        "columns": "unit,value,createdAt",
        "download": "true",
        "format": "csv",
        "from-date": start_date,
        "phenomenon": sensor_title,
        "to-date": end_date
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print(f'Fehler beim Herunterladen der CSV-Datei für {sensor_title}. Statuscode: {response.status_code}')
        return ""


for sensor_title in sensor_titles:
    csv_data = fetch_sensor_data(sensebox_id, sensor_title.get('title'), start_date, end_date)
    if csv_data:
        df = pd.read_csv(io.StringIO(csv_data), usecols=['value', 'createdAt'])
        df = df.rename(columns={'value': sensor_title.get('title')})
        df['createdAt'] = pd.to_datetime(df['createdAt'])
        df['createdAt'] = pd.to_datetime(df.createdAt).dt.tz_localize(None)
        df['createdAt'] = df['createdAt'].dt.floor('min')
        
        # write df to Database. df has columns createdAt, value



print(f'Daten wurden erfolgreich abgerufen und gespeichert.')


Temperature
6252afcfd7e732001bb6b9fb
Global Radiation
6252afcfd7e732001bb6b9fa
Rain (1h)
6252afcfd7e732001bb6b9f9
Windspeed
6252afcfd7e732001bb6b9f8
Humidity
6252bc0fd7e732001bbcacbb
UV-A Radiation
6252bc0fd7e732001bbcacbc
UV-B Radiation
6252bc0fd7e732001bbcacbd
Wind Direction
6415a8fcc383840007fecd71
Pressure
6415b84ec38384000706cb0c
Daten wurden erfolgreich abgerufen und gespeichert.


In [29]:
main_df

,Temperature,createdAt,Global Radiation,Rain (1h),Windspeed,Humidity,UV-A Radiation,UV-B Radiation,Wind Direction,Pressure
0,NaN,2024-06-18 00:00:00,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN
1,NaN,2024-06-18 00:01:00,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,15.34,2024-06-18 00:04:00,1.0,NaN,NaN,90.0,NaN,NaN,NaN,999.9
3,NaN,2024-06-18 00:05:00,NaN,NaN,1.03,NaN,0.00,0.0,100.0,NaN
4,NaN,2024-06-18 00:06:00,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1674,16.67,2024-06-19 21:53:00,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN
1675,NaN,2024-06-19 21:54:00,2.1,NaN,NaN,NaN,0.04,0.0,NaN,1003.6
1676,NaN,2024-06-19 21:55:00,NaN,0.0,0.45,NaN,NaN,NaN,75.0,NaN
1677,16.62,2024-06-19 21:58:00,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN


In [1]:
import requests

In [7]:
import requests
import pandas as pd
import io
import psycopg2
from psycopg2 import sql
from time import time

# Funktion zum Abrufen der Sensortitel und IDs
def get_sensor_titles_and_ids(sensebox_id):
    url = f'https://api.opensensemap.org/boxes/{sensebox_id}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return [{'id': sensor['_id'], 'title': sensor['title']} for sensor in data['sensors']]
    else:
        print(f'Fehler beim Abrufen der Sensor-Titel: {response.status_code}')
        return []

# Funktion zum Abrufen der Sensordaten
def fetch_sensor_data(sensebox_id, sensor_title, start_date, end_date):
    url = "https://api.opensensemap.org//boxes/data"
    params = {
        "boxid": sensebox_id,
        "columns": "unit,value,createdAt",
        "download": "true",
        "format": "csv",
        "from-date": start_date,
        "phenomenon": sensor_title,
        "to-date": end_date
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print(f'Fehler beim Herunterladen der CSV-Datei für {sensor_title}. Statuscode: {response.status_code}')
        return ""

# Datenbank-Verbindungsdetails
dbname = "Umweltmonitoring"
user = "postgres"
password = "1234"
host = "localhost"
port = 5432

sensebox_id = '6252afcfd7e732001bb6b9f7'
start_date = "2024-06-17T00:00:00.000Z"
end_date = "2024-06-19T22:00:00.000Z"
sensor_titles_and_ids = get_sensor_titles_and_ids(sensebox_id)

# Verbindung zur spezifischen Datenbank herstellen
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
conn.autocommit = True
cur = conn.cursor()

# Sensordaten abrufen und in die Datenbank einfügen
for sensor in sensor_titles_and_ids:
    # calculate download time for each sensor
    timestart = time()
    csv_data = fetch_sensor_data(sensebox_id, sensor.get('title'), start_date, end_date)
    if csv_data:
        df = pd.read_csv(io.StringIO(csv_data), usecols=['value', 'createdAt'])
        df = df.rename(columns={'value': sensor.get('title')})
        df['createdAt'] = pd.to_datetime(df['createdAt'])
        df['createdAt'] = pd.to_datetime(df.createdAt).dt.tz_localize(None)
        df['createdAt'] = df['createdAt'].dt.floor('min')
        timeend = time()
        print(f"Download Zeit: {timeend - timestart} Sekunden.")
        # Daten in die Tabelle 'measurement' einfügen
        insert_measurement_query = """
        INSERT INTO measurement (sensor_id, created_at, value) VALUES (%s, %s, %s)
        ON CONFLICT (sensor_id, created_at) DO NOTHING;
        """
        for index, row in df.iterrows():
            cur.execute(insert_measurement_query, (sensor.get('id'), row['createdAt'], row[sensor.get('title')]))

    print(f"Daten für Sensor '{sensor.get('title')}' wurden erfolgreich in die Datenbank eingefügt.")
    

# Verbindung schließen
cur.close()
conn.close()

print(f'\nDaten wurden erfolgreich abgerufen und gespeichert.')


Download Zeit: 1.9182729721069336 Sekunden.
Daten für Sensor 'Temperature' wurden erfolgreich in die Datenbank eingefügt.
Download Zeit: 0.8828411102294922 Sekunden.
Daten für Sensor 'Global Radiation' wurden erfolgreich in die Datenbank eingefügt.
Download Zeit: 1.240441083908081 Sekunden.
Daten für Sensor 'Rain (1h)' wurden erfolgreich in die Datenbank eingefügt.
Download Zeit: 0.8709249496459961 Sekunden.
Daten für Sensor 'Windspeed' wurden erfolgreich in die Datenbank eingefügt.
Download Zeit: 1.2597548961639404 Sekunden.
Daten für Sensor 'Humidity' wurden erfolgreich in die Datenbank eingefügt.
Download Zeit: 1.2411868572235107 Sekunden.
Daten für Sensor 'UV-A Radiation' wurden erfolgreich in die Datenbank eingefügt.
Download Zeit: 1.2331328392028809 Sekunden.
Daten für Sensor 'UV-B Radiation' wurden erfolgreich in die Datenbank eingefügt.
Download Zeit: 1.2386198043823242 Sekunden.
Daten für Sensor 'Wind Direction' wurden erfolgreich in die Datenbank eingefügt.
Download Zeit: 1.1

In [3]:
import requests
def get_sensor_titles_and_ids(sensebox_id):
    url = f'https://api.opensensemap.org/boxes/{sensebox_id}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return [{'id': sensor['_id'], 'title': sensor['title']} for sensor in data['sensors']]
    else:
        print(f'Fehler beim Abrufen der Sensor-Titel: {response.status_code}')
        return []

In [4]:
test = get_sensor_titles_and_ids('6252afcfd7e732001bb6b9f7')

In [5]:
test

[{'id': '6252afcfd7e732001bb6b9fb', 'title': 'Temperature'},
 {'id': '6252afcfd7e732001bb6b9fa', 'title': 'Global Radiation'},
 {'id': '6252afcfd7e732001bb6b9f9', 'title': 'Rain (1h)'},
 {'id': '6252afcfd7e732001bb6b9f8', 'title': 'Windspeed'},
 {'id': '6252bc0fd7e732001bbcacbb', 'title': 'Humidity'},
 {'id': '6252bc0fd7e732001bbcacbc', 'title': 'UV-A Radiation'},
 {'id': '6252bc0fd7e732001bbcacbd', 'title': 'UV-B Radiation'},
 {'id': '6415a8fcc383840007fecd71', 'title': 'Wind Direction'},
 {'id': '6415b84ec38384000706cb0c', 'title': 'Pressure'}]

In [6]:
import psycopg2
import requests
from psycopg2 import sql

# Datenbank-Verbindungsdetails
dbname = "Umweltmonitoring"
user = "postgres"
password = "1234"
host = "localhost"
port = 5432

# Verbindung zur PostgreSQL Datenbank herstellen (default: postgres)
conn = psycopg2.connect(
    dbname="postgres", user=user, password=password, host=host, port=port
)
conn.autocommit = True
cur = conn.cursor()

# Prüfen, ob die Datenbank existiert, und falls nicht, erstellen
cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (dbname,))
exists = cur.fetchone()
if not exists:
    cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(dbname)))

# Verbindung zur spezifischen Datenbank herstellen
cur.close()
conn.close()

conn = psycopg2.connect(
    dbname=dbname, user=user, password=password, host=host, port=port
)
conn.autocommit = True
cur = conn.cursor()

# Tabellen erstellen, falls sie nicht existieren
create_tables_query = """
CREATE TABLE IF NOT EXISTS sensebox (
    sensebox_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(255),
    created_at TIMESTAMP,
    description TEXT,
    exposure VARCHAR(50),
    last_measurement_at TIMESTAMP,
    latitude DECIMAL(9, 6),
    longitude DECIMAL(9, 6),
    altitude DECIMAL(5, 2)
);

CREATE TABLE IF NOT EXISTS sensor (
    sensor_id VARCHAR(50) PRIMARY KEY,
    sensebox_id VARCHAR(50),
    title VARCHAR(255),
    unit VARCHAR(50),
    sensor_type VARCHAR(255),
    icon VARCHAR(255),
    FOREIGN KEY (sensebox_id) REFERENCES sensebox(sensebox_id)
);

CREATE TABLE IF NOT EXISTS measurement (
    measurement_id SERIAL PRIMARY KEY,
    sensor_id VARCHAR(50),
    created_at TIMESTAMP,
    value DECIMAL(10, 2),
    FOREIGN KEY (sensor_id) REFERENCES sensor(sensor_id),
    UNIQUE (sensor_id, created_at)
);
"""
cur.execute(create_tables_query)

# Verbindung schließen
cur.close()
conn.close()

# Sensebox URL
url = "https://api.opensensemap.org/boxes/6252afcfd7e732001bb6b9f7"

# Abrufen der Sensebox-Daten
response = requests.get(url)
sensebox = response.json()

# Verbindung zur spezifischen Datenbank herstellen
conn = psycopg2.connect(
    dbname=dbname, user=user, password=password, host=host, port=port
)
conn.autocommit = True
cur = conn.cursor()

# Sensebox-Daten einfügen
insert_sensebox_query = """
INSERT INTO sensebox (
    sensebox_id, name, created_at, description, exposure, 
    last_measurement_at, latitude, longitude, altitude
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (sensebox_id) DO NOTHING;
"""
sensebox_data = (
    sensebox["_id"],
    sensebox["name"],
    sensebox["createdAt"],
    sensebox["description"],
    sensebox["exposure"],
    sensebox["lastMeasurementAt"],
    sensebox["currentLocation"]["coordinates"][1],
    sensebox["currentLocation"]["coordinates"][0],
    sensebox["currentLocation"]["coordinates"][2],
)
cur.execute(insert_sensebox_query, sensebox_data)

# Sensor-Daten einfügen
insert_sensor_query = """
INSERT INTO sensor (
    sensor_id, sensebox_id, title, unit, sensor_type, icon
) VALUES (%s, %s, %s, %s, %s, %s)
ON CONFLICT (sensor_id) DO NOTHING;
"""
for sensor in sensebox["sensors"]:
    sensor_data = (
        sensor["_id"],
        sensebox["_id"],
        sensor["title"],
        sensor["unit"],
        sensor["sensorType"],
        sensor["icon"],
    )
    cur.execute(insert_sensor_query, sensor_data)

# Verbindung schließen
cur.close()
conn.close()


In [ ]:
import requests
import pandas as pd
import io
import psycopg2
from psycopg2 import sql
from time import time


# Funktion zum Abrufen der Sensortitel und IDs
def get_sensor_titles_and_ids(sensebox_id):
    url = f"https://api.opensensemap.org/boxes/{sensebox_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return [
            {"id": sensor["_id"], "title": sensor["title"]}
            for sensor in data["sensors"]
        ]
    else:
        print(f"Fehler beim Abrufen der Sensor-Titel: {response.status_code}")
        return []


# Funktion zum Abrufen der Sensordaten
def fetch_sensor_data(sensebox_id, sensor_title, start_date, end_date):
    url = "https://api.opensensemap.org//boxes/data"
    params = {
        "boxid": sensebox_id,
        "columns": "unit,value,createdAt",
        "download": "true",
        "format": "csv",
        "from-date": start_date,
        "phenomenon": sensor_title,
        "to-date": end_date,
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.content.decode("utf-8")
    else:
        print(
            f"Fehler beim Herunterladen der CSV-Datei für {sensor_title}. Statuscode: {response.status_code}"
        )
        return ""


# Datenbank-Verbindungsdetails
dbname = "Umweltmonitoring"
user = "postgres"
password = "1234"
host = "localhost"
port = 5432

# Verbindung zur PostgreSQL Datenbank herstellen (default: postgres)
conn = psycopg2.connect(
    dbname=dbname, user=user, password=password, host=host, port=port
)
conn.autocommit = True
cur = conn.cursor()

# Prüfen, ob die Datenbank existiert, und falls nicht, erstellen
cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (dbname,))
exists = cur.fetchone()
if not exists:
    cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(dbname)))

# Verbindung zur spezifischen Datenbank herstellen
cur.close()
conn.close()

conn = psycopg2.connect(
    dbname=dbname, user=user, password=password, host=host, port=port
)
conn.autocommit = True
cur = conn.cursor()

# Tabellen erstellen, falls sie nicht existieren
create_tables_query = """
CREATE TABLE IF NOT EXISTS sensebox (
    sensebox_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(255),
    created_at TIMESTAMP,
    description TEXT,
    exposure VARCHAR(50),
    last_measurement_at TIMESTAMP,
    latitude DECIMAL(9, 6),
    longitude DECIMAL(9, 6),
    altitude DECIMAL(5, 2)
);

CREATE TABLE IF NOT EXISTS sensor (
    sensor_id VARCHAR(50) PRIMARY KEY,
    sensebox_id VARCHAR(50),
    title VARCHAR(255),
    unit VARCHAR(50),
    sensor_type VARCHAR(255),
    icon VARCHAR(255),
    FOREIGN KEY (sensebox_id) REFERENCES sensebox(sensebox_id)
);

CREATE TABLE IF NOT EXISTS measurement (
    measurement_id SERIAL PRIMARY KEY,
    sensor_id VARCHAR(50),
    created_at TIMESTAMP,
    value DECIMAL(10, 2),
    FOREIGN KEY (sensor_id) REFERENCES sensor(sensor_id),
    UNIQUE (sensor_id, created_at)
);
"""
cur.execute(create_tables_query)

# Sensebox URL
url = "https://api.opensensemap.org/boxes/6252afcfd7e732001bb6b9f7"

# Abrufen der Sensebox-Daten
response = requests.get(url)
sensebox = response.json()

# Sensebox-Daten einfügen
insert_sensebox_query = """
INSERT INTO sensebox (
    sensebox_id, name, created_at, description, exposure, 
    last_measurement_at, latitude, longitude, altitude
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (sensebox_id) DO NOTHING;
"""
sensebox_data = (
    sensebox["_id"],
    sensebox["name"],
    sensebox["createdAt"],
    sensebox["description"],
    sensebox["exposure"],
    sensebox["lastMeasurementAt"],
    sensebox["currentLocation"]["coordinates"][1],
    sensebox["currentLocation"]["coordinates"][0],
    sensebox["currentLocation"]["coordinates"][2],
)
cur.execute(insert_sensebox_query, sensebox_data)

# Sensor-Daten einfügen
insert_sensor_query = """
INSERT INTO sensor (
    sensor_id, sensebox_id, title, unit, sensor_type, icon
) VALUES (%s, %s, %s, %s, %s, %s)
ON CONFLICT (sensor_id) DO NOTHING;
"""
for sensor in sensebox["sensors"]:
    sensor_data = (
        sensor["_id"],
        sensebox["_id"],
        sensor["title"],
        sensor["unit"],
        sensor["sensorType"],
        sensor["icon"],
    )
    cur.execute(insert_sensor_query, sensor_data)

# Sensordaten abrufen und in die Datenbank einfügen
start_date = "2024-06-17T00:00:00.000Z"
end_date = "2024-06-19T22:00:00.000Z"
sensor_titles_and_ids = get_sensor_titles_and_ids(sensebox["_id"])

for sensor in sensor_titles_and_ids:
    # calculate download time for each sensor
    timestart = time()
    csv_data = fetch_sensor_data(
        sensebox["_id"], sensor.get("title"), start_date, end_date
    )
    if csv_data:
        df = pd.read_csv(io.StringIO(csv_data), usecols=["value", "createdAt"])
        df = df.rename(columns={"value": sensor.get("title")})
        df["createdAt"] = pd.to_datetime(df["createdAt"])
        df["createdAt"] = pd.to_datetime(df.createdAt).dt.tz_localize(None)
        df["createdAt"] = df["createdAt"].dt.floor("min")
        timeend = time()
        print(f"Download Zeit: {timeend - timestart} Sekunden.")
        # Daten in die Tabelle 'measurement' einfügen
        insert_measurement_query = """
        INSERT INTO measurement (sensor_id, created_at, value) VALUES (%s, %s, %s)
        ON CONFLICT (sensor_id, created_at) DO NOTHING;
        """
        for index, row in df.iterrows():
            cur.execute(
                insert_measurement_query,
                (sensor.get("id"), row["createdAt"], row[sensor.get("title")]),
            )

    print(
        f"Daten für Sensor '{sensor.get('title')}' wurden erfolgreich in die Datenbank eingefügt."
    )

# Verbindung schließen
cur.close()
conn.close()

print(f"\nDaten wurden erfolgreich abgerufen und gespeichert.")
